In [1]:
# pip install geopandas

In [2]:
#pip install folium

In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import matplotlib.mlab as alab
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster 
import warnings
warnings.filterwarnings("ignore")

In [5]:
yellow_cleaned = pd.read_feather('../preprocessed_data/yellow_cleaned.feather').drop('index', axis = 1)

yellow_cleaned.shape

(18539360, 20)

In [ ]:
sf = gpd.read_file('../raw_data/taxi_zones/taxi_zones.shp')
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [ ]:
sf

In [ ]:
sf['geometry']

In [ ]:
pickup_count = yellow_cleaned['PULocationID'].value_counts().sort_index()
pickup_count = pd.DataFrame({'LocationID' : pickup_count.index, 'pickup_count' : pickup_count})
gdf = gpd.GeoDataFrame(pd.merge(pickup_count, sf, left_on = 'LocationID', right_on = 'LocationID')).drop('LocationID', axis = 1)


In [ ]:
gdf

In [ ]:
xmin = 0
xmax = gdf['pickup_count'].max()

fig, ax = plt.subplots(1, figsize = (11, 11))

ax.set_title('Number of piick up at each regin in 3 month')
cb = plt.cm.ScalarMappable(cmap = 'Blues', norm=plt.Normalize(vmin = xmin, vmax = xmax))
cb.set_array([])
fig.colorbar(cb)
gdf.plot(column = 'pickup_count', cmap = 'Blues', linewidth = 1.5, ax = ax, edgecolor = '0.2')

In [ ]:
m = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

folium.Choropleth(
    geo_data = gdf,
    name = 'choropleth,',
    data = gdf,
    columns = ['OBJECTID', 'pickup_count'],
    key_on = 'feature.properties.OBJECTID',
    fill_opacity = 1,
    line_opacity = 0.3,
).add_to(m)

m